In [0]:
# Mount the drive to access the data from drive
from google.colab import drive
drive.mount('/content/drive')

1. Basemodel XGboost - No imputation, but scale_pos
Valid Score = 0.7567241158438351, LBscore =0.6794885791
2. Impute = frequent for missing,Using SMOTE to remove class imbalance, XGBoost with 10 foldCV.
Valid Score =0.9678402054766864,LBscore = 0.668696479543292
3.Impute = frequent, with class weights on,
Valid Score =0.7518523186494807,LB Score=0.6823848280
4. Removing city column to avoid higher dimensions, with 
Valid Score=0.7719847981169748,LBscore=0.67915726879984

In [0]:
!pip install pyforest
!pip install pycaret

In [0]:
!pip install feature_engine

In [0]:
!pip install h2o

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import itertools
import statsmodels.api as sm

import torch
import torch.nn as nn

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
#import pyforest
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE
import xgboost
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier, AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# from feature_engine.categorical_encoders import OneHotCategoricalEncoder
# from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder, OrdinalCategoricalEncoder
# from feature_engine.categorical_encoders import MeanCategoricalEncoder,WoERatioCategoricalEncoder
# from feature_engine.discretisers import DecisionTreeDiscretiser, EqualFrequencyDiscretiser, EqualWidthDiscretiser

#import classification module
#from pycaret.classification import *

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

In [0]:
train = pd.read_csv('/content/drive/My Drive/JantaHack_HR_Analytics/Data/train_jqd04QH.csv')
test = pd.read_csv('/content/drive/My Drive/JantaHack_HR_Analytics/Data/test_KaymcHn.csv')
ss = pd.read_csv('/content/drive/My Drive/JantaHack_HR_Analytics/Data/sample_submission_sxfcbdx.csv')

cols_to_drop = ['enrollee_id']
train = train.drop(cols_to_drop,axis=1)
test = test.drop(cols_to_drop,axis=1)
whole_data = pd.concat([train,test])

In [0]:
whole_data.shape

(33380, 13)

In [0]:
# CV model to find tuned XGBoost model. 
def xgb_train_model(X,y):
  # seed 1301 is highest, 130 gave more than 1301 depth=7, n_estimators=1000
  xgtrain = xgb.DMatrix(X, label=y)
  clf = xgb.XGBClassifier(max_depth = 7,
                  n_estimators=1000,
                  learning_rate=0.01, 
                  nthread=4,
                  subsample=1.0,
                  colsample_bytree=0.5,
                  min_child_weight = 3,
                  seed=130)
  xgb_param = clf.get_xgb_params()
  #do cross validation
  print ('Start cross validation')
  cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=10, metrics=['auc'],
      early_stopping_rounds=50, stratified=True, seed=1301)
  print('Best number of trees = {}'.format(cvresult.shape[0]))
  clf.set_params(n_estimators=cvresult.shape[0])

  print('Fit on the trainingsdata')
  clf.fit(X, y, eval_metric='auc')
  print('Overall AUC:', roc_auc_score(y, clf.predict_proba(X)[:,1]))
  return clf,cvresult

In [0]:
whole_data_pre = whole_data.copy()

# Manually selecting ordinal and nominal categorical columns to do different encodings
nom_cat = ['city','gender','relevent_experience','major_discipline','company_type']
whole_data_pre = pd.get_dummies(whole_data, columns=nom_cat, drop_first=True)

ord_cat = ['education_level','company_size','last_new_job','enrolled_university','experience']

ed_lev = train['education_level'].unique().tolist()[:4] + [train['education_level'].unique().tolist()[5]]
#ed_lev_values = [21,23,17,28,15] # based on age at which the graduation finished(this improved score a bit)
ed_lev_values = [3,4,2,5,1]
ed_lev_map = dict(zip(ed_lev, ed_lev_values))
whole_data_pre['education_level'] = whole_data_pre['education_level'].replace(ed_lev_map)

comp_size = train['company_size'].unique().tolist()[:3] + train['company_size'].unique().tolist()[4:]
comp_size_values = [4,1,3,7,8,6,5,2]
comp_size_map = dict(zip(comp_size, comp_size_values))
whole_data_pre['company_size'] = whole_data_pre['company_size'].replace(comp_size_map)

last_newj = train['last_new_job'].unique().tolist()[:6]
last_newj_values = [1,2,3,5,0,4]
last_newj_map = dict(zip(last_newj, last_newj_values))
whole_data_pre['last_new_job'] = whole_data_pre['last_new_job'].replace(last_newj_map)

enr_uni = train['enrolled_university'].unique().tolist()[:3]
enr_uni_values = [0,2,1]
enr_uni_map = dict(zip(enr_uni, enr_uni_values))
whole_data_pre['enrolled_university'] = whole_data_pre['enrolled_university'].replace(enr_uni_map)

exper = train['experience'].unique().tolist()[:21]+[train['experience'].unique().tolist()[22]]
exper_values = [3,14,6,8,21,4,9,15,10,1,5,16,11,12,7,2,13,0,19,18,17,20]
exper_map = dict(zip(exper,exper_values))
whole_data_pre['experience'] = whole_data_pre['experience'].replace(exper_map)

In [0]:
train_pre = whole_data_pre.iloc[:len(train)]
test_pre = whole_data_pre.iloc[len(train):]
test_pre.drop(['target'],axis=1,inplace=True)

train_pre['no_of_missing'] = train_pre.isnull().sum(axis=1)
train_pre['is_missing'] = np.where(train_pre.isnull().sum(axis=1) == 0, 0,1)

test_pre['no_of_missing'] = test_pre.isnull().sum(axis=1)
test_pre['is_missing'] = np.where(test_pre.isnull().sum(axis=1) == 0, 0,1)

In [0]:
X = train_pre.drop('target',axis=1)
y = train_pre['target']

# define the imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X = imputer.fit_transform(X)
X = pd.DataFrame(X,columns=train_pre.drop('target',axis=1).columns)

test_pre_imp = imputer.transform(test_pre)
test_pre_imp = pd.DataFrame(test_pre_imp,columns= test_pre.columns)

In [0]:
clf,cvresult = xgb_train_model(X,y)
pred = clf.predict_proba(test_pre,ntree_limit=cvresult.shape[0])
ID = pd.read_csv('/content/drive/My Drive/JantaHack_HR_Analytics/Data/test_KaymcHn.csv').enrollee_id
submission = pd.DataFrame({"enrollee_id":ID, "target":pred[:,1]})
name_of_submission = 'Final_Submission.csv'
submission.to_csv(name_of_submission,index=False)

Start cross validation
Best number of trees = 200
Fit on the trainingsdata
Overall AUC: 0.7359366277647874


Important Points
1. Tried lot of categorical encodings and combination of those, but score hasn't been improved from the above model

2.  Different approaches can be using different models like Lightgbm and Catboots, then use stacking on top those to give better score.

3. So concluded that best model has simple imputation(mode) with one hot encoding to nominal categories and label encoding(manually done) for ordinal encodings. Modelling using Xgboost CV.

Note: Class is imbalanced, so choosing cross-validation method is key in this competition

#Thank you for reading this approach